
# Introduction #


We'll see how to stack XGBoost on top of a linear forecaster to make up for XGBoost's inability to extrapolate trends. This and similar combinations are widely used in winning solutions of Kaggle's forecasting competitions.

# Time Series Residuals #

The residuals are what you get when you subtract out the model's predictions during the training period of the target from the target itself -- the residual series is the difference between the actual curve and the fitted curve, in other words.

<img>Residual series, top and bottom, from an earlier example</img>

The residuals are useful because they show you what parts of a time series your model failed to learn a given feature. If the residuals look like noise when you plot them over *time*, you know that the model has learned all of the *time dependence* in the series.

<img>time dependence</img>

And similarly, if the lag plots of the residuals look like noise, you know the model has learned all of the *serial dependence* in the series.

<img>serial dependence</img>

If, however, you detect any patterns or relationships in the residuals, you know your model could still be improved. One way to capture these relationships could be through feature engineering -- by adding more or different seasonal features or indicators for holidays, say.

Another way could be to use a different learning algorithm, one better suited to capturing those relationships; we might want an algorithm capable of learning non-linear relationships, for instance, like XGBoost. But instead of replacing the original model with another (which might have it's own limitations), we can combine the strengths of both through *stacking*.

# Hybrid Forecasters #

We can train a model on the residuals. (This is how GBDTs like XGBoost work, in fact, training decision trees on the residuals of earlier decision trees, over and over again. Which is why you sometimes hear the trick in this section referred to as a "boosting" technique.)

The process looks like this:
- train and predict with first model
- train and predict second model on residuals
- add predictions from second to predictions from first to get final predictions

<img>training on residuals. top: series and fitted curve. middle: residuals and fitted curve. bottom: combined curve</img>

The most common kind of hybrid combines a simple model to learn the main components of individual series with a complex model to learn interactions between series and non-linear effects. We'll combine the linear regression model we've learned about in previous lessons with XGBoost, a powerful decision-tree ensemble of a kind that has dominated tabular competitions on Kaggle.

Ensembles of decision trees (like `RandomForest` and `XGBoost`) excel at capturing nonlinear behavior and interactions. Decision trees, however, make predictions through *interpolation* -- they predict new values through averages. Forecasting, however, is a matter of *extrapolation*, making predictions for data *outside* the range of the training set.

We can overcome this limitation by combining a tree-based model with a trend or seasonal model of the kinds we saw in Lessons 2 and 3. 

<img>top: xgboost failing to model trend, middle: xgboost+hybrid succeeds</img>

Linear regression makes up for XGBoost's lack of extrapolation ability, while XGBoost makes up for linear regression's lack of non-linearity and deep interactions. With stacking, we can have the best of both.

<blockquote>
Some notable hybrid models:
- M4 - exponential smoothing -> neural nets (1st)
- Restaurant (?) - linear regression -> XGBoost (?st)
- (???) - ARIMA -> XGBoost
</blockquote>

# Example - US Retail Sales #

The [*US Retail Sales*](https://www.census.gov/retail/index.html) dataset contains monthly sales data for six retail industries from 1992 to 2019, as collected by the US Census Bureau. Our goal will be to forecast sales in the years 2016-2019 given sales in the earlier years. We'll see how a hybrid linear regression / XGBoost forecaster will outperform XGBoost alone on this dataset.

In [ ]:
#$HIDE_INPUT$
from pathlib import Path
from warnings import simplefilter

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (cross_val_score,
                                     TimeSeriesSplit,
                                     train_test_split)
from statsmodels.tsa.deterministic import (CalendarFourier,
                                           DeterministicProcess)
from xgboost import XGBRegressor


simplefilter("ignore")

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc(
    "figure",
    autolayout=True,
    figsize=(12, 6),
    titlesize=18,
    titleweight='bold',
)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
)

data_dir = Path("../input/ts-course-data/")
industries = ["BuildingMaterials", "Clothing", "FoodAndBeverage"]
retail = pd.read_csv(
    data_dir / "us-retail-sales.csv",
    usecols=['Month'] + industries,
    parse_dates=['Month'],
    index_col='Month',
).to_period('D').reindex(columns=industries)
retail = pd.concat({'Sales': retail}, names=[None, 'Industries'], axis=1)

axs = retail.plot(
    subplots=True,
    sharex=True,
    title="US Retail Sales",
    legend=False,
    ** plot_params,
)
for ax, name in zip(axs, industries):
    ax.legend([name], fontsize='large')

First let's use a linear regression model to learn the trend in each series. For demonstration, we'll use a quadratic (order 2) trend. The code here is just the same as in previous lessons.

In [ ]:
#$HIDE_INPUT$
y = retail.copy()

# Create trend features
dp = DeterministicProcess(
    index=y.index,  # dates from the training data
    constant=True,  # the intercept
    order=2,        # quadratic trend
    drop=True,      # drop terms to avoid collinearity
)
X = dp.in_sample()  # features for the training data

# Test on the years 2016-2019. It will be easier for us later if we
# split the date index instead of the dataframe directly.
idx_train, idx_test = train_test_split(
    y.index, test_size=12 * 4, shuffle=False,
)
X_train, X_test = X.loc[idx_train, :], X.loc[idx_test, :]
y_train, y_test = y.loc[idx_train], y.loc[idx_test]

# Fit trend model
model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)

# Make predictions
y_fit = pd.DataFrame(
    model.predict(X_train),
    index=y_train.index,
    columns=y_train.columns,
)
y_pred = pd.DataFrame(
    model.predict(X_test),
    index=y_test.index,
    columns=y_test.columns,
)

Though the fit isn't perfect, it will be enough for our needs.

In [ ]:
#$HIDE_INPUT$
axs = y_train.plot(color='0.25', subplots=True, sharex=True)
axs = y_test.plot(color='0.25', subplots=True, sharex=True, ax=axs)
axs = y_fit.plot(color='C0', subplots=True, sharex=True, ax=axs)
axs = y_pred.plot(color='C3', subplots=True, sharex=True, ax=axs)
for ax in axs: ax.legend([])
_ = plt.suptitle("Trends")

While the linear regression algorithm is capable of multi-output regression, the XGBoost algorithm is not. To predict multiple series at once with XGBoost, we'll instead convert these series from *wide* format, with one time series per column, to *long* format, with series indexed by categories along rows.

In [ ]:
# The `stack` method converts column labels to row labels, pivoting
# from wide format to long
X = retail.stack()  # pivot dataset wide to long
display(X.head())
y = X.pop('Sales')  # grab target series

So that XGBoost can learn to distinguish our three time series, we'll turn the row labels for `'Industries'` into a categorical feature with a label encoding. We'll also create a feature for annual seasonality by pulling the month numbers out of the time index.

In [ ]:
# Turn row labels into categorical feature columns
X = X.reset_index('Industries')
# Label encoding for 'Industries' feature
for colname in X.select_dtypes(["object", "category"]):
    X[colname], _ = X[colname].factorize()

# Label encoding for annual seasonality
X["Month"] = X.index.month  # 1, 2, ..., 12

# Create splits
X_train, X_test = X.loc[idx_train, :], X.loc[idx_test, :]
y_train, y_test = y.loc[idx_train], y.loc[idx_test]

display(X_train.head())

Now we'll convert the trend predictions made earlier to long format and then subtract them from the original series. That will give us detrended (residual) series that XGBoost can learn.

In [ ]:
# Pivot wide to long (stack) and convert DataFrame to Series (squeeze)
y_fit = y_fit.stack().squeeze()    # trend from training set
y_pred = y_pred.stack().squeeze()  # trend from test set

# Create residuals (the collection of detrended series) from the
# training set
y_resid = y_train - y_fit

# Train XGBoost on the residuals
xgb = XGBRegressor()
xgb.fit(X_train, y_resid)

# Add the predicted residuals onto the predicted trends
y_pred_boosted = xgb.predict(X_train) + y_fit
y_forecast_boosted = xgb.predict(X_test) + y_pred

The fit appears quite good, though we can see how the trend learned by XGBoost is only as good as the trend learned by our linear regression model -- in particular, XGBoost wasn't able to compensate for the poorly fit trend in the `'BuildingMaterials'` series.

In [ ]:
#$HIDE_INPUT$
axs = y_train.unstack(['Industries']).plot(
    color='0.25', subplots=True, sharex=True,
)
axs = y_test.unstack(['Industries']).plot(
    color='0.25', subplots=True, sharex=True, ax=axs,
)
axs = y_pred_boosted.unstack(['Industries']).plot(
    color='C0', subplots=True, sharex=True, ax=axs,
)
axs = y_forecast_boosted.unstack(['Industries']).plot(
    color='C3', subplots=True, sharex=True, ax=axs,
)
for ax in axs: ax.legend([])

Let's compare the performance of our hybrid model with XGBoost when fit directly to the dataset (instead of to a residual series).

In [ ]:
#$HIDE_INPUT$
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
y_pred_xgb = pd.DataFrame(xgb.predict(X_train), index=y_train.index)
y_forecast_xgb = pd.DataFrame(xgb.predict(X_test), index=y_test.index)

axs = y_train.unstack(['Industries']).plot(
    color='0.25', subplots=True, sharex=True,
)
axs = y_test.unstack(['Industries']).plot(
    color='0.25', subplots=True, sharex=True, ax=axs,
)
axs = y_pred_xgb.unstack(['Industries']).plot(
    color='C0', subplots=True, sharex=True, ax=axs,
)
axs = y_forecast_xgb.unstack(['Industries']).plot(
    color='C3', subplots=True, sharex=True, ax=axs,
)
for ax in axs: ax.legend([])

You can see that XGBoost failed to learn the trend in these series completely (adding a trend feature wouldn't help either, in fact). The superior performance of the hybrid method shows up the error metrics as well.

In [ ]:
#$HIDE_INPUT$
from sklearn.metrics import mean_absolute_error

print("Hybrid")
print("------")
print("Train MAE:\t{:.3f}".format(mean_absolute_error(y_train, y_pred_boosted)))
print("Test MAE:\t{:.3f}".format(mean_absolute_error(y_test, y_forecast_boosted)))
print("\n")
print("XGBoost Only")
print("------------")
print("Train MAE:\t{:.3f}".format(mean_absolute_error(y_train, y_pred_xgb)))
print("Test MAE:\t{:.3f}".format(mean_absolute_error(y_test, y_forecast_xgb)))

# Your Turn #